#### This code serves to test Algorithm 2 for the Iterative Domain Information Framework

In [1]:
import glob as glob
import math, random
import pandas as pd
from DatasetNames import get_dataset_class_name
from ExtraFunctions import *

from MultiClassPrediction import MultiClassPrediction
from SingleClassPrediction import SingleClassPrediction

In [2]:
DATASET_NAME = "PathMNIST"
NUMBER_OF_IMAGES = 100 # Total number of images to be used
NUMBER_OF_AUGMENTATIONS = 2 # Number of augmentations to be used
number_of_experiment = 1 # Number of experiment to be conducted

source_directory = f"../Datasets/{DATASET_NAME}/test"  # Change this to your test directory path
_, image_paths = get_balanced_random_images(source_directory, n = NUMBER_OF_IMAGES)

random.seed(55)
random.shuffle(image_paths)
initial_image_paths = image_paths.copy()

dataset_classes_of_interest = get_dataset_class_name(DATASET_NAME)
init_classes_of_interest = list(dataset_classes_of_interest.values())

classes_of_interest_list = [random.sample(init_classes_of_interest, len(init_classes_of_interest)) for i in range(len(image_paths))]

multiclass_image_tasks = create_multiclass_augmented_image_tasks(image_paths, classes_of_interest_list, number_of_classes_to_predict=len(init_classes_of_interest), total_number_of_augmentations=NUMBER_OF_AUGMENTATIONS)

#### Playground

In [3]:
model_list = [
    {
        "model_name": "gpt-4o",
        "model_type": "openai",
    },
    {
        "model_name": "gpt-4o-mini",
        "model_type": "openai",
    }
]

In [4]:
mcp = MultiClassPrediction(batch_size = 5, max_workers=2)
scp = SingleClassPrediction(batch_size = 5, max_workers=2)

In [5]:
exp_results = []

for experiment in range(number_of_experiment):
    print(f"Experiment {experiment+1}/{number_of_experiment}")

    # Iterate over the models
    for model in model_list:
        model_name = model["model_name"]
        model_type = model["model_type"]

        print(f"Model: {model_name}")
        print(f"Model Type: {model_type}")

        # Update the model
        mcp.update_model(model_name)
        scp.update_model(model_name)

        # For each model, initialize the variables for the Algorithm
        n = len(init_classes_of_interest)
        k = 2
        threshold = 2
        number_of_classes_to_predict = math.ceil(n / k)
        image_paths = initial_image_paths
        voting_result_list = None        

        # Update number of classes to predict for the Multiclass Image Tasks
        for multiclass_image in multiclass_image_tasks:
            multiclass_image.set_num_predictions(number_of_classes_to_predict)
            multiclass_image.set_classes(random.sample(init_classes_of_interest, len(init_classes_of_interest)))

        while number_of_classes_to_predict >= threshold:

            # Prompt the model to predict "number_of_classes_to_predict" classes for each image
            rs = mcp.process_images(multiclass_image_tasks, model_type)

            # Set the predicted classes as the new classes of interest
            result_list, image_paths, classes_of_interest_list = extract_image_classes_from_multiclass_predictions(rs, number_of_classes_to_predict)

            print("Image Paths: ", image_paths)
            print("Image Paths: ", len(image_paths))

            log_experiment_results(dataset_classes_of_interest, exp_results, experiment, model_name, model_type, number_of_classes_to_predict, image_paths, classes_of_interest_list)

            # Update the number of classes to predict
            number_of_classes_to_predict = math.ceil(number_of_classes_to_predict / k)

            multiclass_image_tasks = create_new_multitask_image_tasks(number_of_classes_to_predict, result_list)
        if rs:
            # Result from Multiclass Prediction
            result_list, _, _ = extract_image_classes_from_multiclass_predictions(rs)

            # Merge augmentation results
            result_list, voting_result_list = transform_result_list_to_single_image_tasks(result_list)
        else:
            # Go to SingleTask result list directly
            result_list = create_singleclass_image_tasks(image_paths, classes_of_interest_list)

        # Predict the single classes for each image
        f_res = scp.process_images(result_list, model_type)
        correct_classes, pred_classes, image_paths = extract_single_classes(f_res, dataset_classes_of_interest)

        log_experiment_results(dataset_classes_of_interest, exp_results, experiment, model_name, model_type, number_of_classes_to_predict, image_paths, pred_classes, voting_result_list)

Experiment 1/1
Model: gpt-4o
Model Type: openai
Image Paths:  ['../Datasets/PathMNIST/test\\6\\image_7162.png', '../Datasets/PathMNIST/test\\0\\image_887.png', '../Datasets/PathMNIST/test\\1\\image_1228.png', '../Datasets/PathMNIST/test\\4\\image_1596.png', '../Datasets/PathMNIST/test\\3\\image_3284.png', '../Datasets/PathMNIST/test\\1\\image_1703.png', '../Datasets/PathMNIST/test\\1\\image_2657.png', '../Datasets/PathMNIST/test\\8\\image_7038.png', '../Datasets/PathMNIST/test\\2\\image_5067.png', '../Datasets/PathMNIST/test\\2\\image_2461.png', '../Datasets/PathMNIST/test\\5\\image_4007.png', '../Datasets/PathMNIST/test\\4\\image_1596.png', '../Datasets/PathMNIST/test\\5\\image_4007.png', '../Datasets/PathMNIST/test\\2\\image_5067.png', '../Datasets/PathMNIST/test\\6\\image_4176.png', '../Datasets/PathMNIST/test\\0\\image_3423.png', '../Datasets/PathMNIST/test\\7\\image_2935.png', '../Datasets/PathMNIST/test\\2\\image_4415.png', '../Datasets/PathMNIST/test\\4\\image_562.png', '../Data

In [6]:
x_sam = pd.DataFrame(exp_results)

In [7]:
pd.DataFrame(exp_results).to_csv(f"{DATASET_NAME}_results_Algorithm2_EXP1.csv", index=False)